In [1]:
from rl4greencrab.envs.simple_env import SimpleEnv
from stable_baselines3 import PPO, TD3
from sb3_contrib import TQC, RecurrentPPO
from stable_baselines3.common.env_util import make_vec_env
from rl4greencrab.envs.green_crab_monthly_env_norm import greenCrabMonthEnvNormalized
from rl4greencrab import evaluate_agent, multiConstAction, simulator
import pandas as pd
import numpy as np
from rl4greencrab import plot_agent
import ray

INFO: Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO: Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.
2025-08-04 23:47:39,013	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.12/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [2]:
from rl4greencrab import greenCrabMonthEnvSimple, greenCrabMonthEnvSimpleNormalized, greenCrabMonthEnvSize, greenCrabMonthEnvSizeNormalized

In [3]:
config = {
    "w_mort_scale" : 600,
    "growth_k": 0.70,
    'random_start':True,
    'var_penalty_const': 0
    # "curriculum": True
}

In [4]:
# normalized constant action
def normalized_action(act, max_act):
    max_action = max_act
    constant_act = np.array(act)
    return constant_act * 2 / max_action - 1 

In [5]:
act = np.array([100,1,1])
env = greenCrabMonthEnvSimple(config)

In [6]:
normalized_action(act, 3000)

array([-0.93333333, -0.99933333, -0.99933333])

In [12]:
act = np.array([-0.93333333, -0.99933333, -0.99933333])
env = greenCrabMonthEnvSimpleNormalized(config)

In [13]:
env.step(act)

({'crabs': array([-1.], dtype=float32)},
 -0.00288793543809862,
 False,
 False,
 {})

In [8]:
act = np.array([100,1,1])
env = greenCrabMonthEnvSize(config)

In [583]:
act = np.array([0,0,1000])
env = greenCrabMonthEnvSizeNormalized(config)

In [652]:
env.step(act)

({'crabs': array([[-1.        , -1.        , -0.99999994, -0.9999995 , -0.9999997 ,
          -0.99999994, -1.        , -1.        , -1.        , -1.        ,
          -1.        , -1.        , -1.        , -1.        , -1.        ,
          -1.        , -1.        , -1.        , -1.        , -1.        ,
          -1.        ]], dtype=float32)},
 -200.28006129272586,
 False,
 False,
 {})